# 🎯 SD1.5 Prompting Techniques Experiment
## Exploring Advanced Prompting for Jewelry Generation

Based on [Machine Learning Mastery's Prompting Techniques for Stable Diffusion](https://machinelearningmastery.com/prompting-techniques-stable-diffusion/)

### Experiment Structure:
- **8 Jewelry Prompts** from Arcade Assignment
- **7 Prompting Techniques** + Baseline
- **6 Seeds per technique** for consistency testing
- **Grid Layout**: 8 rows (prompts) × 7 columns (baseline + 6 techniques)

### Techniques to Test:
1. **Baseline** - Simple prompt
2. **Medium Specification** - Photography/digital art medium
3. **Artistic Style** - Adding art movement styles
4. **Artist Names** - Famous artist influences
5. **Resolution & Detail** - 4K, sharp focus, highly detailed
6. **Lighting** - Cinematic, rim lighting, volumetric
7. **Keyword Emphasis** - Using weights and brackets


In [ ]:
# 🔧 Setup and Configuration
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuration
MODEL_ID = "runwayml/stable-diffusion-v1-5"
DEVICE = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"
OUTPUT_DIR = "sd15_prompting_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Seeds for consistency
SEEDS = [42, 123, 256, 512, 789, 1024]

print(f"🔧 Using device: {DEVICE}")
print(f"📁 Output directory: {OUTPUT_DIR}")
print(f"🎲 Using seeds: {SEEDS}")

# Load SD1.5 pipeline
print("🚀 Loading Stable Diffusion 1.5...")
pipe = StableDiffusionPipeline.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if DEVICE != "cpu" else torch.float32,
    safety_checker=None,
    requires_safety_checker=False
)

# Use DPM++ 2M Karras sampler (similar to article)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to(DEVICE)

print("✅ Pipeline loaded successfully!")


In [ ]:
# 💎 Jewelry Prompts from Arcade Assignment
BASE_PROMPTS = [
    "channel-set diamond eternity band, 2 mm width, hammered 18k yellow gold, product-only white background",
    "Tiffany-style solitaire engagement ring, round brilliant diamond, 18k white gold prong setting, product-only white background", 
    "vintage Art Deco sapphire ring, emerald-cut center stone, diamond side stones, platinum setting, product-only white background",
    "men's wedding band, brushed finish, 6mm width, 14k white gold, comfort fit, product-only white background",
    "halo engagement ring, oval diamond center, pavé halo, rose gold setting, product-only white background",
    "tennis bracelet, round brilliant diamonds, 18k yellow gold setting, 7 inch length, product-only white background",
    "pearl drop earrings, South Sea pearls, 18k gold hooks, classic design, product-only white background",
    "stackable rings set, three thin bands, mixed metals, diamonds and gemstones, product-only white background"
]

# Standard negative prompt from the article
NEGATIVE_PROMPT = """(worst quality, low quality, normal quality, low-res, low details, oversaturated, undersaturated, overexposed, underexposed, grayscale, bw, bad photo, bad photography, bad art:1.4), (watermark, signature, text font, username, error, logo, words, letters, digits, autograph, trademark, name:1.2), (blur, blurry, grainy), morbid, ugly, asymmetrical, mutated malformed, mutilated, poorly lit, bad shadow, draft, cropped, out of frame, cut off, censored, jpeg artifacts, out of focus, glitch, duplicate, (airbrushed, cartoon, anime, semi-realistic, CGI, render, blender, digital art, manga, amateur:1.3), (3D,3D Game, 3D Game Scene, 3D Character:1.1), (bad hands, bad anatomy, bad body, bad face, bad teeth, bad arms, bad legs, deformities:1.3)"""

# Generation settings (based on article recommendations)
GENERATION_PARAMS = {
    "num_inference_steps": 20,
    "guidance_scale": 7.0,
    "width": 512,
    "height": 512,
}

print("💎 Loaded 8 jewelry prompts")
print("🚫 Standard negative prompt configured") 
print("⚙️ Generation parameters set")
print(f"📏 Image size: {GENERATION_PARAMS['width']}x{GENERATION_PARAMS['height']}")
print(f"🎛️ CFG Scale: {GENERATION_PARAMS['guidance_scale']}")
print(f"📈 Steps: {GENERATION_PARAMS['num_inference_steps']}")


In [ ]:
# 🛠️ Helper Functions for Image Generation and Display

def generate_images_for_technique(prompts, technique_name, seeds=SEEDS):
    """Generate images for all prompts using a specific technique"""
    print(f"\n🎨 Generating images for: {technique_name}")
    results = {}
    
    for i, prompt in enumerate(prompts):
        print(f"  📝 Prompt {i+1}/8: {prompt[:50]}...")
        prompt_images = []
        
        for j, seed in enumerate(seeds):
            print(f"    🎲 Seed {j+1}/{len(seeds)}: {seed}")
            
            # Set seed for reproducibility
            generator = torch.Generator(device=DEVICE).manual_seed(seed)
            
            # Generate image
            try:
                with torch.no_grad():
                    result = pipe(
                        prompt=prompt,
                        negative_prompt=NEGATIVE_PROMPT,
                        generator=generator,
                        **GENERATION_PARAMS
                    )
                    image = result.images[0]
                    prompt_images.append(image)
                    
                    # Save individual image
                    filename = f"{technique_name}_p{i+1:02d}_s{seed}.png"
                    image.save(os.path.join(OUTPUT_DIR, filename))
                    
            except Exception as e:
                print(f"      ❌ Error: {e}")
                # Create placeholder image
                placeholder = Image.new('RGB', (512, 512), color='red')
                prompt_images.append(placeholder)
        
        results[i] = prompt_images
    
    return results

def create_prompt_text_image(prompt, width=512, height=512):
    """Create an image with prompt text for the grid"""
    img = Image.new('RGB', (width, height), color='white')
    draw = ImageDraw.Draw(img)
    
    try:
        # Try to use a nice font
        font = ImageFont.truetype('/System/Library/Fonts/Arial.ttf', 14)
    except:
        # Fallback to default font
        font = ImageFont.load_default()
    
    # Wrap text to fit the image
    words = prompt.split()
    lines = []
    current_line = []
    
    for word in words:
        current_line.append(word)
        test_line = ' '.join(current_line)
        bbox = draw.textbbox((0, 0), test_line, font=font)
        if bbox[2] - bbox[0] > width - 20:  # 10px padding on each side
            if len(current_line) > 1:
                current_line.pop()
                lines.append(' '.join(current_line))
                current_line = [word]
            else:
                lines.append(word)
                current_line = []
    
    if current_line:
        lines.append(' '.join(current_line))
    
    # Draw text centered
    y_offset = (height - len(lines) * 20) // 2
    for i, line in enumerate(lines):
        bbox = draw.textbbox((0, 0), line, font=font)
        text_width = bbox[2] - bbox[0]
        x_offset = (width - text_width) // 2
        draw.text((x_offset, y_offset + i * 20), line, fill='black', font=font)
    
    return img

def display_technique_grid(all_results, technique_names):
    """Display all results in a comprehensive grid"""
    num_prompts = len(BASE_PROMPTS)
    num_techniques = len(technique_names)
    
    # Create figure
    fig, axes = plt.subplots(num_prompts, num_techniques + 1, 
                           figsize=(4 * (num_techniques + 1), 3 * num_prompts))
    
    # Ensure axes is 2D
    if num_prompts == 1:
        axes = axes.reshape(1, -1)
    
    fig.suptitle('SD1.5 Prompting Techniques Comparison', fontsize=16, fontweight='bold')
    
    for prompt_idx in range(num_prompts):
        # First column: prompt text
        prompt_img = create_prompt_text_image(BASE_PROMPTS[prompt_idx])
        axes[prompt_idx, 0].imshow(prompt_img)
        axes[prompt_idx, 0].set_title(f'Prompt {prompt_idx + 1}', fontsize=10, fontweight='bold')
        axes[prompt_idx, 0].axis('off')
        
        # Other columns: technique results (using first seed for display)
        for tech_idx, technique_name in enumerate(technique_names):
            col_idx = tech_idx + 1
            if technique_name in all_results and prompt_idx in all_results[technique_name]:
                # Show the first generated image (seed 0)
                image = all_results[technique_name][prompt_idx][0]
                axes[prompt_idx, col_idx].imshow(image)
            else:
                # Placeholder
                axes[prompt_idx, col_idx].text(0.5, 0.5, 'No Image', 
                                             ha='center', va='center', transform=axes[prompt_idx, col_idx].transAxes)
            
            axes[prompt_idx, col_idx].set_title(technique_name.replace('_', ' ').title(), fontsize=10)
            axes[prompt_idx, col_idx].axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'technique_comparison_grid.png'), dpi=150, bbox_inches='tight')
    plt.show()

print("🛠️ Helper functions loaded:")


## 🎯 Technique 1: Baseline (Simple Prompts)
**What it is**: Using the original jewelry prompts without modifications  
**From Article**: "The core of a prompt lies in describing the details of the expected image"  
**Purpose**: Establish baseline performance for comparison


In [ ]:
# 🎯 Technique 1: Baseline Generation
baseline_prompts = BASE_PROMPTS.copy()

print("📝 Baseline prompts (unchanged):")
for i, prompt in enumerate(baseline_prompts):
    print(f"  {i+1}. {prompt}")

# Generate images
baseline_results = generate_images_for_technique(baseline_prompts, "baseline")

print(f"\n✅ Baseline generation complete!")
print(f"📸 Generated {len(baseline_results)} sets of {len(SEEDS)} images each")


## 📸 Technique 2: Medium Specification
**What it is**: Adding photography/artistic medium to prompts  
**From Article**: "Adding the medium of image creation makes the prompt even more specific"  
**Examples**: "Ultra-realistic photography", "Portrait digital painting", "Concept art"  
**For Jewelry**: Professional product photography terminology


In [ ]:
# 📸 Technique 2: Medium Specification
medium_prompts = []

for prompt in BASE_PROMPTS:
    # Add professional product photography medium
    enhanced_prompt = f"Professional jewelry photography of {prompt}, studio lighting, macro lens, commercial product photography"
    medium_prompts.append(enhanced_prompt)

print("📸 Medium-enhanced prompts:")
for i, prompt in enumerate(medium_prompts):
    print(f"  {i+1}. {prompt}")

# Generate images
medium_results = generate_images_for_technique(medium_prompts, "medium_spec")

print(f"\n✅ Medium specification generation complete!")
print(f"📸 Generated {len(medium_results)} sets of {len(SEEDS)} images each")


## 🎨 Technique 3: Artistic Style
**What it is**: Adding artistic movement/style keywords  
**From Article**: "Keywords such as modernist, impressionist, pop art, surrealist, art nouveau, hyperrealistic add an artistic angle"  
**Examples**: "hyperrealistic", "minimalist", "art nouveau"  
**For Jewelry**: Styles that complement luxury product aesthetics


In [ ]:
# 🎨 Technique 3: Artistic Style
artistic_prompts = []

for prompt in BASE_PROMPTS:
    # Add artistic style keywords suitable for luxury jewelry
    enhanced_prompt = f"Hyperrealistic minimalist {prompt}, elegant aesthetic, luxury product design, clean composition"
    artistic_prompts.append(enhanced_prompt)

print("🎨 Artistic style prompts:")
for i, prompt in enumerate(artistic_prompts):
    print(f"  {i+1}. {prompt}")

# Generate images
artistic_results = generate_images_for_technique(artistic_prompts, "artistic_style")

print(f"\n✅ Artistic style generation complete!")
print(f"📸 Generated {len(artistic_results)} sets of {len(SEEDS)} images each")


## 👨‍🎨 Technique 4: Artist Names
**What it is**: Adding famous artist names to influence style  
**From Article**: "Adding artist names picks the style of the artist. Multiple artist names can be mentioned to combine their styles"  
**Examples**: Artists known for precision, luxury, and detailed work  
**For Jewelry**: Artists known for detailed, luxurious, or geometric work


In [ ]:
# 👨‍🎨 Technique 4: Artist Names  
artist_prompts = []

for prompt in BASE_PROMPTS:
    # Add artist names known for precision and luxury aesthetics
    enhanced_prompt = f"{prompt}, by Peter Halley and James Jean, luxury product photography style, precise geometric forms"
    artist_prompts.append(enhanced_prompt)

print("👨‍🎨 Artist-influenced prompts:")
for i, prompt in enumerate(artist_prompts):
    print(f"  {i+1}. {prompt}")

# Generate images
artist_results = generate_images_for_technique(artist_prompts, "artist_names")

print(f"\n✅ Artist names generation complete!")
print(f"📸 Generated {len(artist_results)} sets of {len(SEEDS)} images each")


## 🔍 Technique 5: Resolution & Detail Enhancement
**What it is**: Adding resolution and detail keywords  
**From Article**: "Adding resolution specifications such as highly detailed, HD, 4K, 8K, vray, unreal engine, or sharp focus helps get much more details"  
**Examples**: "4K", "sharp focus", "highly detailed", "macro photography"  
**For Jewelry**: Emphasizing fine details and crystal-clear quality


In [ ]:
# 🔍 Technique 5: Resolution & Detail Enhancement
detail_prompts = []

for prompt in BASE_PROMPTS:
    # Add resolution and detail enhancement keywords
    enhanced_prompt = f"{prompt}, 4K, highly detailed, sharp focus, macro photography, crystal clear, ultra-detailed"
    detail_prompts.append(enhanced_prompt)

print("🔍 Detail-enhanced prompts:")
for i, prompt in enumerate(detail_prompts):
    print(f"  {i+1}. {prompt}")

# Generate images
detail_results = generate_images_for_technique(detail_prompts, "detail_enhanced")

print(f"\n✅ Detail enhancement generation complete!")
print(f"📸 Generated {len(detail_results)} sets of {len(SEEDS)} images each")


## 💡 Technique 6: Lighting Enhancement
**What it is**: Adding specific lighting keywords  
**From Article**: "Adding lighting keywords can enhance the look and feel of the scene. Examples include rim lighting, cinematic lighting, volumetric lighting, crepuscular rays, backlight, or dimly lit"  
**Examples**: "rim lighting", "cinematic lighting", "soft lighting"  
**For Jewelry**: Professional jewelry photography lighting techniques


In [ ]:
# 💡 Technique 6: Lighting Enhancement
lighting_prompts = []

for prompt in BASE_PROMPTS:
    # Add professional jewelry photography lighting
    enhanced_prompt = f"{prompt}, cinematic lighting, soft rim lighting, professional jewelry lighting, luminous, pristine lighting setup"
    lighting_prompts.append(enhanced_prompt)

print("💡 Lighting-enhanced prompts:")
for i, prompt in enumerate(lighting_prompts):
    print(f"  {i+1}. {prompt}")

# Generate images
lighting_results = generate_images_for_technique(lighting_prompts, "lighting_enhanced")

print(f"\n✅ Lighting enhancement generation complete!")
print(f"📸 Generated {len(lighting_results)} sets of {len(SEEDS)} images each")


## ⚖️ Technique 7: Keyword Emphasis & Weighting
**What it is**: Using parentheses and weights to emphasize important terms  
**From Article**: "We can modify a keyword's weightage using the syntax (keyword: factor). The factor is the numeric value."  
**Examples**: "(diamond: 1.2)", "((luxury))", "[cheap: 0.8]"  
**For Jewelry**: Emphasizing key materials, quality, and luxury aspects


In [ ]:
# ⚖️ Technique 7: Keyword Emphasis & Weighting
emphasis_prompts = []

for prompt in BASE_PROMPTS:
    # Add keyword emphasis to important jewelry terms
    if "diamond" in prompt.lower():
        enhanced_prompt = prompt.replace("diamond", "(diamond:1.3)")
    else:
        enhanced_prompt = prompt
    
    if "gold" in enhanced_prompt.lower():
        enhanced_prompt = enhanced_prompt.replace("gold", "(gold:1.2)")
    
    # Add general emphasis on luxury and quality
    enhanced_prompt = f"{enhanced_prompt}, ((luxury)), (premium quality:1.2), (exquisite craftsmanship:1.1)"
    emphasis_prompts.append(enhanced_prompt)

print("⚖️ Emphasis-weighted prompts:")
for i, prompt in enumerate(emphasis_prompts):
    print(f"  {i+1}. {prompt}")

# Generate images
emphasis_results = generate_images_for_technique(emphasis_prompts, "keyword_emphasis")

print(f"\n✅ Keyword emphasis generation complete!")
print(f"📸 Generated {len(emphasis_results)} sets of {len(SEEDS)} images each")


## 📊 Final Results: Technique Comparison Grid
### 8 Rows (Prompts) × 7 Columns (Techniques) Display
Showing all generated images in a comprehensive comparison grid


In [ ]:
# 📊 Compile All Results and Display Grid
all_results = {
    "baseline": baseline_results,
    "medium_spec": medium_results,
    "artistic_style": artistic_results,
    "artist_names": artist_results,
    "detail_enhanced": detail_results,
    "lighting_enhanced": lighting_results,
    "keyword_emphasis": emphasis_results
}

technique_names = list(all_results.keys())

print("📊 COMPREHENSIVE RESULTS SUMMARY")
print("=" * 50)
for technique, results in all_results.items():
    print(f"✅ {technique.replace('_', ' ').title()}: {len(results)} prompts × {len(SEEDS)} seeds = {len(results) * len(SEEDS)} total images")

print(f"\n🎯 Total images generated: {len(all_results) * len(BASE_PROMPTS) * len(SEEDS)}")

# Display the comprehensive comparison grid
print("\n🎨 Displaying technique comparison grid...")
display_technique_grid(all_results, technique_names)

print(f"\n💾 All individual images saved to: {OUTPUT_DIR}")
print(f"📈 Grid comparison saved as: {OUTPUT_DIR}/technique_comparison_grid.png")


## 🎯 Analysis & Insights
### Comparing Prompting Techniques for Jewelry Generation

**Key Observations to Look For:**
1. **Detail Quality**: Which techniques produce sharper, more detailed jewelry?
2. **Material Accuracy**: How well does each technique represent metals and gems?
3. **Lighting Effects**: Which approaches create the most appealing lighting?
4. **Consistency**: How consistent are results across different seeds?
5. **Style Coherence**: Does the technique maintain jewelry aesthetics?

**From the Machine Learning Mastery Article:**
- Simple prompts establish baseline but may lack specificity
- Medium specification helps model understand the desired output format
- Artistic styles can enhance aesthetic appeal but may override realism
- Artist names provide stylistic direction but results can be unpredictable
- Resolution keywords significantly improve detail quality
- Lighting terms are crucial for product photography
- Keyword emphasis allows fine-tuned control over important elements

**Next Steps:**
1. Analyze the grid results to identify best-performing techniques
2. Combine successful elements from multiple techniques
3. Create optimized prompts for production use
4. Test hybrid approaches mixing the most effective elements
